# 1. Loading Data

In [1]:
import sys
import gc
import os
import warnings
import pickle
import statsmodels.api as sm
from pylab import rcParams
import time
from  datetime import datetime, timedelta

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, metrics

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

register_matplotlib_converters()
sns.set()

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# # Google Colab trick to extend memory
# a = []
# while(1):
#     a.append('1')


## 1.1 Functions

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def display_missing(df):    
    for col in df.columns.tolist():  
        if df[col].isnull().sum() != 0:
            print('{} column missing values: {}'.format(col, df[col].isnull().sum()))
    print('\n')
    


## 1.2 Loading data grid


In [3]:
# Mount google drive
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

In [4]:
# Setting directories where data is stored and ouptut dir
if IN_COLAB:
    DATA_GRID_INPUT_DIR = './drive/My Drive/Colab Notebooks' 
    DATA_OUTPUT_DIR = './drive/My Drive/Colab Notebooks'
    !ls './drive/My Drive/Colab Notebooks'
else:
    DATA_GRID_INPUT_DIR = '.'
    DATA_OUTPUT_DIR = '.'

In [ ]:
print('Loading the data...')

data = pd.read_pickle(f'{DATA_GRID_INPUT_DIR}/m5_data_model2.pkl')

Loading the data...


## 1.3 Init variables

In [ ]:
h = 28 # Prediction horizon
max_lags = 120 # Max lags used
TRAINING_LAST_DAY_NUM = 1913 # Last day for training data
FIRST_PRED_DAY = datetime(2016,4, 25) # First prediction day
FIRST_LOADING_DAY = datetime(2013, 4,7) # First day for training
FIRST_LOADING_DAY_NUM = 800
SEED = 7


# 2. Feature Engineering

## Creating features


In [ ]:
# data = data.loc[data.date > '2015-01-01'] 

In [ ]:
# data

In [ ]:
# dept_store_df = data[['id', 'date', 'item_id', 'dept_id', 'store_id', 'sales']].groupby(['dept_id', 'store_id', 'date'])['sales'].sum().reset_index()

In [ ]:
# dept_store_df

In [ ]:
# dept_store_df['dept_store_lag_3'] = dept_store_df.groupby(['dept_id', 'store_id'])['sales'].shift(28)
# dept_store_df['dept_sotre_rmean_3_3'] = dept_store_df.groupby(['dept_id', 'store_id'])['dept_store_lag_3'].transform(lambda x: x.rolling(3).mean())
# dept_store_df.drop(['sales'], axis=1, inplace=True)

In [ ]:
# dept_store_df.loc[dept_store_df.date=='2016-04-23']

In [ ]:
# data = data.merge(dept_store_df, on=['dept_id', 'store_id', 'date'], copy=False)

In [ ]:
# data.loc[data.date=='2016-04-23']

In [ ]:
# Init global variable to store columns of encodings
# To use them later for test dataset

# target_enc_cols = []

In [ ]:
def create_features(df):

#     agg_levels = [['dept_id', 'store_id'],
#                   ['item_id']
#                   ]
#     agg_level_names = ['_'.join(level) for level in agg_levels]
    
#     # Create dataframes grouped by agg_levels and date
#     agg_df = dict()
#     for level, level_name in zip(agg_levels, agg_level_names):
#         agg_df[level_name] = df[['id', 'date', 'item_id', 'dept_id', 'store_id', 'sales']].groupby(level + ['date'])['sales'].sum().reset_index()

    lags = [1, 7, 14, 28]
    lag_cols = [f"lag_t{lag}" for lag in lags ]
  
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[["id","sales"]].groupby("id")["sales"].shift(lag).astype(np.float16)
        
#         # Setting lags for agg levels
#         if lag in [1, 7, 28]:
#             for level, level_name in zip(agg_levels, agg_level_names):
#                 agg_df[level_name][level_name + '_' + lag_col] = agg_df[level_name].groupby(level)['sales'].shift(lag)

    wins = [7, 14, 28, 60]
    for win in wins:
        for lag, lag_col in zip(lags, lag_cols):
            df[f"rmean_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()).astype(np.float16)

            df[f"rmedian_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).median()).astype(np.float16)
            # df[f"rdiff_mean_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).diff().mean()).astype(np.float16)
            df[f"rstd_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).std()).astype(np.float16)
            df[f'rmean_{lag}_{win}_decay'] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x: x.ewm(span=win).mean()).astype(np.float16)

#             # Computing rollings for aggregation levels
#             if (lag in [1, 7, 28]) and (win in [7, 28]):
#                 for level, level_name in zip(agg_levels, agg_level_names):
#                     agg_df[level_name][level_name + '_' + f'rmean_{lag}_{win}'] = agg_df[level_name].groupby(level)[level_name + '_' + lag_col].transform(lambda x: x.rolling(win).mean())
            
            

#     # Merging agg levels computations with main grid
#     for level, level_name in zip(agg_levels, agg_level_names):
#         agg_df[level_name].drop(['sales'], axis=1, inplace=True)
#         df = df.merge(agg_df[level_name], on=level + ['date'], copy=False)

    df['price_mean_t60'] = df[['id','sell_price']].groupby(["id"])["sell_price"].transform(lambda x: x.rolling(60).mean()).astype(np.float16)
    df['price_momentum_t60'] = (df['sell_price'] / df['price_mean_t60']).astype(np.float16)
    
        
    
    # Adding mean/std target encoding features

    # Columns for to encode
#     icols =  [
#             ['state_id'],
#             ['store_id'],
#             ['cat_id'],
#             ['dept_id'],
#             ['state_id', 'cat_id'],
#             ['state_id', 'dept_id'],
#             ['store_id', 'cat_id'],
#             ['store_id', 'dept_id'],
#             ['item_id'],
#             ['item_id', 'state_id'],
#             ['item_id', 'store_id']
#             ]

#     global target_enc_cols 
    
#     for col in icols:
#         print('Encoding', col)
#         # TODO: Make this with variable, or may be use d column as integer
#         temp_df = df[df['date'] < datetime(2016, 3, 28)] # to be sure we don't have leakage in our validation set

#         temp_df = temp_df.groupby(col).agg({'sales': ['std','mean']})
#         col_name = '_enc_'+'_'.join(col)+'_'
#         new_columns = [col_name.join(col).strip() for col in temp_df.columns.values]
#         temp_df.columns = new_columns
#         temp_df = temp_df.reset_index()
#         #print(temp_df)
#         df = df.merge(temp_df, on=col, how='left')
#         #print(df)
#         # Save columns for later usage
#         target_enc_cols += new_columns
#         del temp_df
#         gc.collect()
    
    date_features = {
        
        "wday": "weekday",
        "woy": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     df.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in df.columns:
            df[date_feat_name] = df[date_feat_name].astype("int16")
        else:
            df[date_feat_name] = getattr(df["date"].dt, date_feat_func).astype("int16")
            
    
    return df
    

In [ ]:
%%time

data = create_features(data)

In [ ]:
# Choose one day from data set, to have
# all encodings for every id for later merge 
# with test data

# mean_encodings_df = data.loc[data['d'] == 'd_1913', ['id'] + target_enc_cols].copy()

In [ ]:
data.info()

In [ ]:
data.dropna(inplace = True)
data.shape

In [ ]:
data

## Reduce mem usage of created features

In [ ]:
data = reduce_mem_usage(data)

In [ ]:
gc.collect()

# 3. Fit & Predict

In [ ]:
print('Data usage: {} GB'.format(data.memory_usage().sum() / 10**9))
data.head()

In [ ]:
# train_end_dt = datetime(2016, 3, 27)
# valid_end_dt = datetime(2016, 4, 24)

valid_start = datetime(2016, 3, 28)
train_valid_end_dt = datetime(2016, 4, 27)

In [ ]:
%%time

cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", "weights"] + \
                ['lag_t1'] + ['dept_id_store_id_lag_t1', 'item_id_lag_t1'] # lag_t1 leads to overfitting
train_cols = data.columns[~data.columns.isin(useless_cols)]
#used_cols = train_cols.append(pd.Index(['weights'])) # with weights

# Splitting train and validation by date (28 days before prediction horizon)
# To drop na values only from training set
# train = data.loc[data.date <= train_end_dt].dropna()
X_train = data[train_cols]
y_train = data["sales"]

X_valid= data.loc[(data.date >= valid_start) & (data.date <=train_valid_end_dt), train_cols]
y_valid = data.loc[(data.date >= valid_start) & (data.date <= train_valid_end_dt), "sales"]

del data
gc.collect()

X_train_np = X_train.values.astype(np.float16)
X_valid_np = X_valid.values.astype(np.float16)

del X_train, X_valid
gc.collect()

train_data = lgb.Dataset(X_train_np, label = y_train, feature_name = list(train_cols), categorical_feature=cat_feats, free_raw_data=False)
valid_data = lgb.Dataset(X_valid_np, label = y_valid, feature_name = list(train_cols), categorical_feature=cat_feats, free_raw_data=False)
# train_data = lgb.Dataset(X_train[train_cols], label = y_train, weight=X_train['weights'], categorical_feature=cat_feats, free_raw_data=False)
# valid_data = lgb.Dataset(X_valid[train_cols], label = y_valid, weight=X_valid['weights'], categorical_feature=cat_feats, free_raw_data=False)


# Random train-validation split
# X_train = data[used_cols]
# y_train = data["sales"]

# np.random.seed(SEED)
# valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
# train_inds = np.setdiff1d(X_train.index.values, valid_inds)

# train_data = lgb.Dataset(X_train.loc[train_inds, train_cols] ,\
#                          label = y_train.loc[train_inds], weight=X_train.loc[train_inds, "weights"],\
#                          categorical_feature=cat_feats, free_raw_data=False)
# valid_data = lgb.Dataset(X_train.loc[valid_inds, train_cols], \
#                          label = y_train.loc[valid_inds], weight=X_train.loc[valid_inds, "weights"],\
#                          categorical_feature=cat_feats, free_raw_data=False)
# del valid_inds, train_inds

# del X_train, X_valid



In [ ]:
params = {
#             'device' : 'gpu', # Need for local GPU computations
#             'max_bin': 31, # For better GPU performance
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'subsample': 0.5,
            'subsample_freq': 1,
            'learning_rate': 0.03,
            'num_leaves': 2**11-1,
            'min_data_in_leaf': 2**12-1,
            'feature_fraction': 0.5,
            'max_bin': 100,
            'n_estimators': 1400,
            'boost_from_average': False,
            'verbose': 1,
            'n_jobs': 4, # For local computation optimization
            'seed': SEED,
} 

# params = {
#         "objective" : "poisson",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
# #         "sub_feature" : 0.8,
#         "sub_row" : 0.8,
#         "bagging_freq" : 1,
#         'feature_fraction': 0.8,
#         "lambda_l2" : 0.1,
# #         "nthread" : 4
#         'verbosity': 1,
#         'num_iterations' : 1200,
#         'num_leaves': 2**7-1,
#         "min_data_in_leaf": 2**7-1,
#         'early_stopping_rounds': 125,
#         'seed': SEED,
# }

In [ ]:
# # Trick to reduce memory spike while model starts training
# train_data.save_binary(f'{DATA_OUTPUT_DIR}/train.bin')
# valid_data.save_binary(f'{DATA_OUTPUT_DIR}/valid.bin')
# del train_data, valid_data
# gc.collect()
# train_data = lgb.Dataset(f'{DATA_OUTPUT_DIR}/train.bin', categorical_feature=cat_feats, two_round=True)
# valid_data = lgb.Dataset(f'{DATA_OUTPUT_DIR}/valid.bin', categorical_feature=cat_feats, two_round=True)

In [ ]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [train_data, valid_data], 
                  verbose_eval=50)


In [ ]:
os.system('say "Training complete"')

In [ ]:
m_lgb.save_model(f'{DATA_OUTPUT_DIR}/model.lgb')
m_lgb = lgb.Booster(model_file=f'{DATA_OUTPUT_DIR}/model.lgb')

In [ ]:
feature_importance = pd.DataFrame({"Value": m_lgb.feature_importance("gain"), "Feature": m_lgb.feature_name()}) \
                    .sort_values(by="Value", ascending=False)

# Change size of the plot, so we can see all features
fig_dims = (10, 14)
fig, ax = plt.subplots(figsize=fig_dims)

sns.barplot(x="Value", y="Feature", ax=ax, data=feature_importance)
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()

In [ ]:
# Detection of features with zero-importance
zero_features = list(feature_importance[feature_importance['Value'] == 0]['Feature'])
print('\nThere are {} features with 0.0 importance'.format(len(zero_features)))
print(zero_features)
feature_importance

In [ ]:
%%time 

tdata = pd.read_pickle(f'{DATA_GRID_INPUT_DIR}/m5_data_test_model2.pkl')

In [ ]:
def create_lag_features_for_test(df, day):
    
#     agg_levels = [['dept_id', 'store_id'],
#                   ['item_id']
#                   ]
#     agg_level_names = ['_'.join(level) for level in agg_levels]
    
#     # Create dataframes grouped by agg_levels and date
#     agg_df = dict()
#     for level, level_name in zip(agg_levels, agg_level_names):
#         agg_df[level_name] = df[['id', 'date', 'item_id', 'dept_id', 'store_id', 'sales']].groupby(level + ['date'])['sales'].sum().reset_index()
      
    # create lag feaures just for single day (faster)
    lags = [1, 7, 14, 28]
    lag_cols = [f"lag_t{lag}" for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        df.loc[df.date == day, lag_col] = df.loc[df.date ==day-timedelta(days=lag), 'sales'].values  # !!! main
        
#         # Setting lags for agg levels
#         if lag in [1, 7, 28]:
#             for level, level_name in zip(agg_levels, agg_level_names):
#                 agg_df[level_name][level_name + '_' + lag_col] = agg_df[level_name].groupby(level)['sales'].shift(lag)

    wins = [7, 14, 28, 60]
    for win in wins:
        for lag, lag_col in zip(lags, lag_cols):
            df_win = df[(df.date <= day-timedelta(days=lag)) & (df.date > day-timedelta(days=lag+win))]
            df_win_grouped_mean = df_win.groupby("id").agg({'sales':'mean'}).reindex(df.loc[df.date==day,'id'])
            df.loc[df.date == day,f"rmean_{lag}_{win}"] = df_win_grouped_mean.sales.values            

            df_win_grouped_median = df_win.groupby("id").agg({'sales':'median'}).reindex(df.loc[df.date==day,'id'])
            df.loc[df.date == day,f"rmedian_{lag}_{win}"] = df_win_grouped_median.sales.values
            df_win_grouped_std = df_win.groupby("id").agg({'sales':'std'}).reindex(df.loc[df.date==day,'id'])
            df.loc[df.date == day,f"rstd_{lag}_{win}"] = df_win_grouped_std.sales.values

            df[f'rmean_{lag}_{win}_decay'] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x: x.ewm(span=win).mean()).astype(np.float16)
#             df[f"rmedian_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).median()).astype(np.float16)
#             df[f"rstd_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).std()).astype(np.float16)

#             # Computing rollings for aggregation levels
#             if (lag in [1, 7, 28]) and (win in [7, 28]):
#                 for level, level_name in zip(agg_levels, agg_level_names):
#                     agg_df[level_name][level_name + '_' + f'rmean_{lag}_{win}'] = agg_df[level_name].groupby(level)[level_name + '_' + lag_col].transform(lambda x: x.rolling(win).mean())


    
    
    # Merging agg levels computations with main grid
    for level, level_name in zip(agg_levels, agg_level_names):
        agg_df[level_name].drop(['sales'], axis=1, inplace=True)
        df = df.merge(agg_df[level_name], on=level + ['date'], copy=False)    

    return df
    
    
## Creating features for test data
def create_static_features_for_test(df):
    # We create lags here, so we can use them later 
    # for weighted moving average computations
    lags = [1, 7, 14, 28]
    lag_cols = [f"lag_t{lag}" for lag in lags ]

    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[["id","sales"]].groupby("id")["sales"].shift(lag).astype(np.float16)
    
    # copy of the code from `create_df()` above
    date_features = {
        "wday": "weekday",
        "woy": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
    }

    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in df.columns:
            df[date_feat_name] = df[date_feat_name].astype("int16")
        else:
            df[date_feat_name] = getattr(
                df["date"].dt, date_feat_func).astype("int16")
            
    # Create price features
    df['price_mean_t60'] = df[['id','sell_price']].groupby(["id"])["sell_price"].transform(lambda x: x.rolling(60).mean()).astype(np.float16)
    df['price_momentum_t60'] = (df['sell_price'] / df['price_mean_t60']).astype(np.float16)
    
    # Add mean encoding features
#     global mean_encodings_df
#     df = df.merge(mean_encodings_df, on=['id'])

    return df


In [ ]:
tdata = create_static_features_for_test(tdata)

In [ ]:
# # FOR TEST
# day = FIRST_PRED_DAY + timedelta(days=0)
# print(i, day)
# tst = tdata[(tdata.date >= day - timedelta(days=max_lags)) & (tdata.date <= day)].copy()
# create_lag_features_for_test(tst, day)
# tst = tst.loc[tst.date == day, train_cols]


In [ ]:
# os.system("say 'Task complete'")

In [ ]:
# tst[tst.isna().any(axis=1)].shape[0] > 0

In [ ]:
%%time

for i in range(0, 28):
    day = FIRST_PRED_DAY + timedelta(days=i)
    print(i, day)
    tst = tdata[(tdata.date >= day - timedelta(days=max_lags)) & (tdata.date <= day)].copy()
    tst = create_lag_features_for_test(tst, day)
    tst = tst.loc[tst.date == day, train_cols]
    # Check that all features generated correctly
    if tst[tst.isna().any(axis=1)].shape[0] > 0:
        print('Some values in tst are nans:')
        print(tst[tst.isna().any(axis=1)])
    tdata.loc[tdata.date == day, "sales"] = m_lgb.predict(tst.values.astype(np.float32)) # 1.035*


In [ ]:
os.system('say "Prediction complete"')

In [ ]:
tdata.loc[(tdata.date >= FIRST_PRED_DAY) & (tdata.sales > 2)].count()

In [ ]:
%%time

tdata_sub = tdata.loc[tdata.date >= FIRST_PRED_DAY, ["id", "sales"]].copy()
tdata_sub.loc[tdata.date >= FIRST_PRED_DAY+ timedelta(days=h), "id"] = tdata_sub.loc[tdata.date >= FIRST_PRED_DAY+timedelta(days=h), 
                                                                     "id"].str.replace("validation$", "evaluation")
tdata_sub["F"] = [f"F{rank}" for rank in tdata_sub.groupby("id")["id"].cumcount()+1]
tdata_sub = tdata_sub.set_index(["id", "F" ]).unstack()["sales"][[f"F{i}" for i in range(1,29)]].reset_index()
tdata_sub.fillna(0., inplace = True)

# # kyakovlev magic trick
# for i in range(1,29):
#     tdata_sub['F'+str(i)] *= 1.02

tdata_sub.to_csv(f"{DATA_OUTPUT_DIR}/submission.csv",index=False)
tdata_sub.shape


In [ ]:
tst